In [1]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import spacy
import re 
import pandas as pd
import matplotlib.pyplot as plt

## Load Data

In [4]:
bbc_data = pd.read_csv("bbc_news.csv")
bbc_data.head()

,Unnamed: 0,index,title,pubDate,guid,link,description
0,0,6684,Can I refuse to work?,"Wed, 10 Aug 2022 15:46:18 GMT",https://www.bbc.co.uk/news/business-62147992,https://www.bbc.co.uk/news/business-62147992?a...,With much of the UK enduring another period of...
1,1,9267,'Liz Truss the Brief?' World reacts to UK poli...,"Mon, 17 Oct 2022 11:35:12 GMT",https://www.bbc.co.uk/news/world-63285480,https://www.bbc.co.uk/news/world-63285480?at_m...,The UK's political chaos has been watched arou...
2,2,7387,Rationing energy is nothing new for off-grid c...,"Wed, 31 Aug 2022 05:20:18 GMT",https://www.bbc.co.uk/news/uk-scotland-highlan...,https://www.bbc.co.uk/news/uk-scotland-highlan...,Scoraig in the north west Highlands has long h...
3,3,767,The hunt for superyachts of sanctioned Russian...,"Tue, 22 Mar 2022 14:37:01 GMT",https://www.bbc.co.uk/news/60739336,https://www.bbc.co.uk/news/60739336?at_medium=...,"Wealthy Russians sanctioned by the US, EU and ..."
4,4,3712,Platinum Jubilee: 70 years of the Queen in 70 ...,"Wed, 01 Jun 2022 23:17:33 GMT",https://www.bbc.co.uk/news/uk-61660128,https://www.bbc.co.uk/news/uk-61660128?at_medi...,A quick look back at the Queen's 70 years on t...


In [5]:
bbc_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Unnamed: 0   1000 non-null   int64 
 1   index        1000 non-null   int64 
 2   title        1000 non-null   object
 3   pubDate      1000 non-null   object
 4   guid         1000 non-null   object
 5   link         1000 non-null   object
 6   description  1000 non-null   object
dtypes: int64(2), object(5)
memory usage: 54.8+ KB


In [15]:
# titlles = pd.Series(bbc_data['title'])
titles = pd.DataFrame(bbc_data['title'])
titles.head()

,title
0,Can I refuse to work?
1,'Liz Truss the Brief?' World reacts to UK poli...
2,Rationing energy is nothing new for off-grid c...
3,The hunt for superyachts of sanctioned Russian...
4,Platinum Jubilee: 70 years of the Queen in 70 ...


## Clean data

In [17]:
# lower case
titles['lowercase'] = titles['title'].str.lower()
titles.head()

,title,lowercase
0,can i refuse to work?,can i refuse to work?
1,'liz truss the brief?' world reacts to uk poli...,'liz truss the brief?' world reacts to uk poli...
2,rationing energy is nothing new for off-grid c...,rationing energy is nothing new for off-grid c...
3,the hunt for superyachts of sanctioned russian...,the hunt for superyachts of sanctioned russian...
4,platinum jubilee: 70 years of the queen in 70 ...,platinum jubilee: 70 years of the queen in 70 ...


In [18]:
# remove stopwords
stop_words = stopwords.words('english')
titles['no_stopwords'] = titles['lowercase'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))
titles.head()

,title,lowercase,no_stopwords
0,can i refuse to work?,can i refuse to work?,refuse work?
1,'liz truss the brief?' world reacts to uk poli...,'liz truss the brief?' world reacts to uk poli...,'liz truss brief?' world reacts uk political t...
2,rationing energy is nothing new for off-grid c...,rationing energy is nothing new for off-grid c...,rationing energy nothing new off-grid community
3,the hunt for superyachts of sanctioned russian...,the hunt for superyachts of sanctioned russian...,hunt superyachts sanctioned russian oligarchs
4,platinum jubilee: 70 years of the queen in 70 ...,platinum jubilee: 70 years of the queen in 70 ...,platinum jubilee: 70 years queen 70 seconds


In [19]:
# punctation removal
titles['no_stopwords_no_punct'] = titles.apply(lambda x: re.sub(r"([^\w\s])", "", x['no_stopwords']), axis=1)
titles.head()

,title,lowercase,no_stopwords,no_stopwords_no_punct
0,can i refuse to work?,can i refuse to work?,refuse work?,refuse work
1,'liz truss the brief?' world reacts to uk poli...,'liz truss the brief?' world reacts to uk poli...,'liz truss brief?' world reacts uk political t...,liz truss brief world reacts uk political turmoil
2,rationing energy is nothing new for off-grid c...,rationing energy is nothing new for off-grid c...,rationing energy nothing new off-grid community,rationing energy nothing new offgrid community
3,the hunt for superyachts of sanctioned russian...,the hunt for superyachts of sanctioned russian...,hunt superyachts sanctioned russian oligarchs,hunt superyachts sanctioned russian oligarchs
4,platinum jubilee: 70 years of the queen in 70 ...,platinum jubilee: 70 years of the queen in 70 ...,platinum jubilee: 70 years queen 70 seconds,platinum jubilee 70 years queen 70 seconds


In [20]:
# tokenize
titles['tokens_raw'] = titles.apply(lambda x: word_tokenize(x['title']), axis=1)
titles['tokens_clean'] = titles.apply(lambda x: word_tokenize(x['no_stopwords_no_punct']), axis=1)
titles.head()

,title,lowercase,no_stopwords,no_stopwords_no_punct,tokens_raw,tokens_clean
0,can i refuse to work?,can i refuse to work?,refuse work?,refuse work,"[can, i, refuse, to, work, ?]","[refuse, work]"
1,'liz truss the brief?' world reacts to uk poli...,'liz truss the brief?' world reacts to uk poli...,'liz truss brief?' world reacts uk political t...,liz truss brief world reacts uk political turmoil,"['liz, truss, the, brief, ?, ', world, reacts,...","[liz, truss, brief, world, reacts, uk, politic..."
2,rationing energy is nothing new for off-grid c...,rationing energy is nothing new for off-grid c...,rationing energy nothing new off-grid community,rationing energy nothing new offgrid community,"[rationing, energy, is, nothing, new, for, off...","[rationing, energy, nothing, new, offgrid, com..."
3,the hunt for superyachts of sanctioned russian...,the hunt for superyachts of sanctioned russian...,hunt superyachts sanctioned russian oligarchs,hunt superyachts sanctioned russian oligarchs,"[the, hunt, for, superyachts, of, sanctioned, ...","[hunt, superyachts, sanctioned, russian, oliga..."
4,platinum jubilee: 70 years of the queen in 70 ...,platinum jubilee: 70 years of the queen in 70 ...,platinum jubilee: 70 years queen 70 seconds,platinum jubilee 70 years queen 70 seconds,"[platinum, jubilee, :, 70, years, of, the, que...","[platinum, jubilee, 70, years, queen, 70, seco..."


In [21]:
# lemmatizing 
lemmatizer = WordNetLemmatizer()
titles["tokens_clean_lemmatized"] = titles["tokens_clean"].apply(lambda tokens: [lemmatizer.lemmatize(token) for token in tokens])
titles.head()

,title,lowercase,no_stopwords,no_stopwords_no_punct,tokens_raw,tokens_clean,tokens_clean_lemmatized
0,can i refuse to work?,can i refuse to work?,refuse work?,refuse work,"[can, i, refuse, to, work, ?]","[refuse, work]","[refuse, work]"
1,'liz truss the brief?' world reacts to uk poli...,'liz truss the brief?' world reacts to uk poli...,'liz truss brief?' world reacts uk political t...,liz truss brief world reacts uk political turmoil,"['liz, truss, the, brief, ?, ', world, reacts,...","[liz, truss, brief, world, reacts, uk, politic...","[liz, truss, brief, world, reacts, uk, politic..."
2,rationing energy is nothing new for off-grid c...,rationing energy is nothing new for off-grid c...,rationing energy nothing new off-grid community,rationing energy nothing new offgrid community,"[rationing, energy, is, nothing, new, for, off...","[rationing, energy, nothing, new, offgrid, com...","[rationing, energy, nothing, new, offgrid, com..."
3,the hunt for superyachts of sanctioned russian...,the hunt for superyachts of sanctioned russian...,hunt superyachts sanctioned russian oligarchs,hunt superyachts sanctioned russian oligarchs,"[the, hunt, for, superyachts, of, sanctioned, ...","[hunt, superyachts, sanctioned, russian, oliga...","[hunt, superyachts, sanctioned, russian, oliga..."
4,platinum jubilee: 70 years of the queen in 70 ...,platinum jubilee: 70 years of the queen in 70 ...,platinum jubilee: 70 years queen 70 seconds,platinum jubilee 70 years queen 70 seconds,"[platinum, jubilee, :, 70, years, of, the, que...","[platinum, jubilee, 70, years, queen, 70, seco...","[platinum, jubilee, 70, year, queen, 70, second]"


In [23]:
# create lists for just our tokens
tokens_raw_list = sum(titles['tokens_raw'], []) #unpack our lists into a single list
tokens_clean_list = sum(titles['tokens_clean_lemmatized'], [])
print(f"Raw tokens: {tokens_raw_list}")
print(f"Clean tokens: {tokens_clean_list}")

Raw tokens: ['can', 'i', 'refuse', 'to', 'work', '?', "'liz", 'truss', 'the', 'brief', '?', "'", 'world', 'reacts', 'to', 'uk', 'political', 'turmoil', 'rationing', 'energy', 'is', 'nothing', 'new', 'for', 'off-grid', 'community', 'the', 'hunt', 'for', 'superyachts', 'of', 'sanctioned', 'russian', 'oligarchs', 'platinum', 'jubilee', ':', '70', 'years', 'of', 'the', 'queen', 'in', '70', 'seconds', 'red', 'bull', 'found', 'guilty', 'of', 'breaking', 'formula', '1', "'s", 'budget', 'cap', 'world', 'triathlon', 'championship', 'series', ':', 'flora', 'duffy', 'beats', 'georgia', 'taylor-brown', 'to', 'women', "'s", 'title', 'terry', 'hall', ':', 'coventry', 'scooter', 'ride-out', 'pays', 'tribute', 'to', 'singer', 'post', 'office', 'and', 'fujitsu', 'to', 'face', 'inquiry', 'over', 'horizon', 'scandal', "'pavement", 'parking', 'frightens', 'me', "'", 'uk', 'interest', 'rates', ':', 'how', 'will', 'the', 'rise', 'affect', 'you', 'and', 'how', 'high', 'could', 'it', 'go', '?', 'they', 'staye

## POS tagging

In [24]:
nlp = spacy.load("en_core_web_sm")
doc = nlp(" ".join(tokens_raw_list))

pos_df = pd.DataFrame(columns=["token", "post_tag"])

for token in doc:
    pos_df = pd.concat(
        [pos_df, pd.DataFrame.from_records([{"token": token.text, "pos_tag": token.pos_}])],
        ignore_index=True
    )


pos_df.head(15)

,token,post_tag,pos_tag
0,can,NaN,AUX
1,i,NaN,PRON
2,refuse,NaN,VERB
3,to,NaN,PART
4,work,NaN,VERB
5,?,NaN,PUNCT
6,',NaN,PUNCT
7,liz,NaN,PROPN
8,truss,NaN,VERB
9,the,NaN,DET


In [25]:
pos_df_counts = pos_df.groupby(["token", "pos_tag"]).size().reset_index(name="counts").sort_values(by="counts", ascending=False)
pos_df_counts.head(10)

,token,pos_tag,counts
96,:,PUNCT,543
8,',PUNCT,283
3734,the,DET,226
1842,in,ADP,192
3783,to,PART,177
2594,of,ADP,173
21,-,PUNCT,165
220,and,CCONJ,149
15,'s,PART,143
98,?,PUNCT,130


In [35]:
nouns = pos_df_counts[pos_df_counts.pos_tag == 'NOUN'][0:10]
nouns

,token,pos_tag,counts
4171,world,NOUN,39
4051,war,NOUN,39
2263,man,NOUN,26
940,cup,NOUN,26
4159,women,NOUN,19
2826,police,NOUN,18
2699,papers,NOUN,18
4083,week,NOUN,17
4194,year,NOUN,17
1243,energy,NOUN,17


In [36]:
verbs = pos_df_counts[pos_df_counts.pos_tag == 'VERB'][0:10]
verbs

,token,pos_tag,counts
3239,says,VERB,30
3916,ukraine,VERB,19
1472,found,VERB,13
9,',VERB,13
4136,win,VERB,9
1559,get,VERB,9
4146,wins,VERB,9
2035,killed,VERB,8
3660,take,VERB,8
3238,say,VERB,8


## NER 

In [38]:
ner_df = pd.DataFrame(columns=["token", "ner_tag"])

for ent in doc.ents:
    if pd.isna(ent.label_) is False:
        ner_df = pd.concat(
            [ner_df, pd.DataFrame.from_records([{"token": ent.text, "ner_tag": ent.label_}])],
            ignore_index=True
        )

ner_df.head(15)

,token,ner_tag
0,superyachts,CARDINAL
1,russian,NORP
2,70 years,DATE
3,70 seconds,TIME
4,bull,PERSON
5,1,CARDINAL
6,georgia taylor-brown,PERSON
7,terry hall,PERSON
8,fujitsu,ORG
9,uk,GPE


In [39]:
ner_df_counts = ner_df.groupby(["token", "ner_tag"]).size().reset_index(name="counts").sort_values(by="counts", ascending=False)
ner_df_counts.head(10)

,token,ner_tag,counts
37,2022,CARDINAL,31
468,russian,NORP,22
214,england,GPE,21
550,uk,GPE,21
227,first,ORDINAL,17
551,ukraine,GPE,13
234,france,GPE,11
467,russia,GPE,10
38,2022,DATE,10
164,china,GPE,10


In [40]:
people = ner_df_counts[ner_df_counts.ner_tag == 'PERSON'][0:10]
people

,token,ner_tag,counts
446,putin,PERSON,7
137,boris johnson,PERSON,6
260,harry,PERSON,4
321,jurgen klopp,PERSON,3
583,zelensky,PERSON,3
107,andy murray,PERSON,3
111,antonio conte,PERSON,2
266,hodgkinson,PERSON,2
456,rishi,PERSON,2
128,beth,PERSON,2
